# Mission - Classifiez automatiquement des informations

Vous êtes mandaté en tant que Consultant Data Scientist par le département RH de votre client. Il s’agit de l'ESN TechNova Partners, spécialisée dans le conseil en transformation digitale et la vente d’applications en SaaS.
 
Ils font face à un turnover plus élevé que d'habitude et ils souhaitent identifier les causes racines potentielles derrière ces démissions.

### Importation des librairies

In [26]:
# Librairies "classiques"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from IPython.display import Image, display

# Librairies scikit-learn
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
# Preprocess et modèles
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

### On reprend les éléments du notebook de l'étape 2

In [4]:
amelioration_modele = pd.read_csv("../Data/Processed/Projet_4_etape2_clean.csv")

# Partie 4 - Améliorez l'approche de classification

### Recommandation 1 - Avant toute implémentation en code, mettre sur le papier votre approche de modélisation. C’est-à-dire : avec quelle méthodologie je découpe mon jeu de données en apprentissage et test, de quelles métriques d’évaluation je vais avoir besoin etc.

* L'objectif va être de gérer le déséquilibre de notre jeu de données. Nous devons également mieux le stratifier et déterminer le bon seuil de classification avec une courbe PR. Nous sommes dans une situation de déséquilibre alors la courbe ROC n'est pas adaptée.

* Utilisation d'un calcul avec F1 pour déterminer les bons seuils
* D'après les résultats obtenus par l'étape 3, il semble important de mettre l'accent sur l'amélioration de la métrique Recall qui n'est pas bonne pour chacun des modèles testés.
* D'autant plus dans un sens métier, il est plus judicieux de bien trouver les Faux Négatifs, c'est à dire les salariés qui ont démissionné mais qui sont considérés comme encore dans l'entreprise.
* La métrique Recall et la matrice de confusion vont nous permettre de mieux comprendre les résultats.
* Utilisation également de la moyenne et de l'écart-type pour juger de la performance.
* La métrique Accuracy n'est pas utilisable pour le moment car le déséquilibre fausse la métrique, elle va trouver facilement le Non qui domine notre jeu de données à hauteur de 84%.

Réalisation de différents tests sur 3 modèles :
* Choix du modèle linéaire LogisticRegression
* Choix du modèle non-linéaire XGBoost
* Choix du modèle non-linéaire RandomForest

## Création d'une fonction pour appliquer nos modèles

#### Colonnes à scaler ou déjà encodées


In [27]:
features_a_scaler = [
    'revenu_mensuel','annee_experience_totale','annees_dans_l_entreprise','distance_domicile_travail',
    'annees_depuis_la_derniere_promotion','experience_externe','score_satisfaction',
    'augmentation_par_formation','pee_par_anciennete','niveau_education']
features_encodees = [
    'genre','heure_supplementaires',
    'frequence_deplacement','a_suivi_formation','tranche_age','statut_marital_Celibataire',
    'statut_marital_Divorce','statut_marital_Marie','promotion_recente','poste_AssistantdeDirection',
    'poste_CadreCommercial','poste_Consultant','poste_DirecteurTechnique','poste_Manager',
    'poste_ReprésentantCommercial','poste_RessourcesHumaines','poste_SeniorManager','poste_TechLead']

In [58]:
# Création d'une fonction et pour figer nos conditions
def modelisation(model, amelioration_modele, target_col='a_quitte_l_entreprise', test_size=0.2, random_state=42):
    y = amelioration_modele['a_quitte_l_entreprise']
    X = amelioration_modele.drop(columns='a_quitte_l_entreprise')

# Test en jeu d'apprentissage et de test
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=test_size, random_state=random_state)
    
 # Demande d'une transformation des colonnes non encodées en scaling et création d'un pipeline
    preprocessor = ColumnTransformer(
        transformers = [
            ('num',StandardScaler(),features_a_scaler),
            ('cat','passthrough',features_encodees)
        ])
    pipeline = Pipeline(steps=[
        ('preproccesor', preprocessor),
        ('model', model)
    ])
# Choix des indicateurs de performance et création de la validation croisée    
    scoring = ['precision','recall','f1','average_precision'] # average_precision = PR AUC
    cv_results = cross_validate(
        pipeline,
        X_train, y_train, # validation croisée que sur l'entraînement
        cv=StratifiedKFold(n_splits=3), #KFold va renvoyer des plis stratifiés
        scoring=scoring,
        return_train_score=True,
        n_jobs=-1
    )
    print("=== Résultats CV (train vs val) ===")

# Boucle pour calcul de nos indicateurs
    for metric in scoring:
        tr = cv_results[f"train_{metric}"]
        te = cv_results[f"test_{metric}"]
        print(f"{metric:18s}: train {tr.mean():.3f} ± {tr.std():.3f} vs val {te.mean():.3f} ± {te.std():.3f}")


# Entraînement du modèle
    pipeline.fit(X_train,y_train)

# Prédictions sur tain et test
    y_pred_train = pipeline.predict(X_train)
    y_pred_test = pipeline.predict(X_test)
    proba_test = pipeline.predict_proba(X_test)[:,1]
    
    print("\n=== Classification Report — TRAIN ===")
    print(classification_report(y_train, y_pred_train, digits=3))
    print("=== Classification Report — TEST ===")
    print(classification_report(y_test, y_pred_test, digits=3))

# Calcul Average Precision sur Test + le seuil optimal
    ap_test = average_precision_score(y_test, proba_test)
    prec, rec, thr = precision_recall_curve(y_test, proba_test)

    print(f"\nPR AUC (Average Precision) — TEST : {ap_test:.3f}")

    return model

#### Résultats pour la régression logistique

In [59]:
modelisation(LogisticRegression(), amelioration_modele)

=== Résultats CV (train vs val) ===
precision         : train 0.801 ± 0.009 vs val 0.719 ± 0.084
recall            : train 0.376 ± 0.025 vs val 0.348 ± 0.069
f1                : train 0.512 ± 0.025 vs val 0.463 ± 0.060
average_precision : train 0.662 ± 0.016 vs val 0.600 ± 0.048

=== Classification Report — TRAIN ===
              precision    recall  f1-score   support

           0      0.886     0.982     0.931       978
           1      0.804     0.374     0.510       198

    accuracy                          0.879      1176
   macro avg      0.845     0.678     0.721      1176
weighted avg      0.872     0.879     0.860      1176

=== Classification Report — TEST ===
              precision    recall  f1-score   support

           0      0.897     0.961     0.928       255
           1      0.524     0.282     0.367        39

    accuracy                          0.871       294
   macro avg      0.711     0.621     0.647       294
weighted avg      0.848     0.871     0.854  

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


#### Résultats pour le RandomForest

In [63]:
modelisation(RandomForestClassifier(), amelioration_modele)

=== Résultats CV (train vs val) ===
precision         : train 1.000 ± 0.000 vs val 0.777 ± 0.104
recall            : train 1.000 ± 0.000 vs val 0.247 ± 0.052
f1                : train 1.000 ± 0.000 vs val 0.368 ± 0.044
average_precision : train 1.000 ± 0.000 vs val 0.558 ± 0.042

=== Classification Report — TRAIN ===
              precision    recall  f1-score   support

           0      1.000     1.000     1.000       978
           1      1.000     1.000     1.000       198

    accuracy                          1.000      1176
   macro avg      1.000     1.000     1.000      1176
weighted avg      1.000     1.000     1.000      1176

=== Classification Report — TEST ===
              precision    recall  f1-score   support

           0      0.875     0.988     0.928       255
           1      0.500     0.077     0.133        39

    accuracy                          0.867       294
   macro avg      0.688     0.533     0.531       294
weighted avg      0.825     0.867     0.823  

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


#### Résultats pour le XGBoost

In [64]:
modelisation(XGBClassifier(), amelioration_modele)

=== Résultats CV (train vs val) ===
precision         : train 1.000 ± 0.000 vs val 0.609 ± 0.030
recall            : train 1.000 ± 0.000 vs val 0.379 ± 0.093
f1                : train 1.000 ± 0.000 vs val 0.457 ± 0.061
average_precision : train 1.000 ± 0.000 vs val 0.533 ± 0.017

=== Classification Report — TRAIN ===
              precision    recall  f1-score   support

           0      1.000     1.000     1.000       978
           1      1.000     1.000     1.000       198

    accuracy                          1.000      1176
   macro avg      1.000     1.000     1.000      1176
weighted avg      1.000     1.000     1.000      1176

=== Classification Report — TEST ===
              precision    recall  f1-score   support

           0      0.887     0.953     0.919       255
           1      0.400     0.205     0.271        39

    accuracy                          0.854       294
   macro avg      0.643     0.579     0.595       294
weighted avg      0.822     0.854     0.833  

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None
